In [7]:
import sqlite3
from mt.definitions import DATA_DIR

In [10]:
def list_all_tables(cursor):
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return [row[0] for row in cursor.fetchall()]

def print_table_contents(cursor, table_name):
    cursor.execute(f"SELECT * FROM {table_name};")
    rows = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]
    
    print(f"\nTable: {table_name}")
    print(f"Columns: {', '.join(column_names)}")
    # for row in rows:
    #     print(row)

def view_database_contents(database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    
    tables = list_all_tables(cursor)
    print(tables)
    for table in tables:
        print_table_contents(cursor, table)
    
    conn.close()

# Path to the SQLite database
database_path = DATA_DIR / 'test.db'

# View database contents
view_database_contents(database_path)


['metadata', 'graphs', 'file_paths', 'root_paths']

Table: metadata
Columns: version

Table: graphs
Columns: file, tag, error, value

Table: file_paths
Columns: file, local_id, value

Table: root_paths
Columns: file, symbol_stack, value


In [15]:
import sqlite3
import networkx as nx

def export_stack_graph(database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    graph_data = {
        'nodes': {},
        'edges': []
    }

    # Fetch file paths to map file IDs to paths
    file_paths = {}
    cursor.execute("SELECT file, value FROM file_paths")
    for row in cursor.fetchall():
        file, value = row
        file_paths[file] = value

    # Fetch nodes and edges from the 'graphs' table
    cursor.execute("SELECT file, tag, error, value FROM graphs")
    for row in cursor.fetchall():
        print(row)
        file, tag, error, value = row
        if tag == 'node':
            node_id = value
            graph_data['nodes'][node_id] = {'file': file_paths.get(file, file)}
        elif tag == 'edge':
            source, target = value.split('->')
            graph_data['edges'].append((source, target, error))
    
    conn.close()
    return graph_data

def convert_to_networkx(graph_data):
    G = nx.DiGraph()
    
    # Add nodes to the graph
    for node_id, attributes in graph_data['nodes'].items():
        G.add_node(node_id, **attributes)
    
    # Add edges to the graph
    for edge in graph_data['edges']:
        source, target, label = edge
        G.add_edge(source, target, label=label)
    
    return G


# Export stack graph data from the SQLite database
stack_graph_data = export_stack_graph(database_path)

# Convert to NetworkX graph
G = convert_to_networkx(stack_graph_data)

# Print NetworkX graph info
print(G)


('/home/fox/projects/example_data_crawler/example_data_crawler/__init__.py', 'zO2hbZ7yAeL6WymB6V8prdo37WA', None, b'\x01H/home/fox/projects/example_data_crawler/example_data_crawler/__init__.pyu\x06\x00\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x01\x00\x02\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x07\x01H/home/fox/projects/example_data_crawler/example_data_crawler/__init__.py\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x03\x0ctsg_location\x11line 43 column 14\x0ctsg_variable\x11@node.after_scope\x0etsg_match_node\x1b[syntax node module (1, 1)]\x07\x01H/home/fox/projects/example_data_crawler/example_data_crawler/__init__.py\x01\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x03\x0ctsg_variable\x12@node.before_scope\x0ctsg_location\x11line 44 column 14\x0etsg_match_node\x1b[syntax node module (1, 1)]\x03\x01H/home/fox/projects/example_d